In [1]:
import keras_cv
import numpy as np

Using TensorFlow backend


In [2]:
base_model = tf.keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
	input_tensor=Input(shape=(640, 640, 3))
)

UnimplementedError: Exception encountered when calling layer 'conv1_conv' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} DNN library is not found. [Op:Conv2D] name: 

Call arguments received by layer 'conv1_conv' (type Conv2D):
  • inputs=tf.Tensor(shape=(1, 512, 512, 3), dtype=float32)

In [ ]:
flatten = base_model.output
flatten = Flatten()(flatten)

In [ ]:
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid",name="boxes")(bboxHead)

In [ ]:
softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(len(class_mapping), activation="softmax",name="classes")(softmaxHead)

In [ ]:
custom_model = Model(
	inputs=base_model.input,
	outputs=(bboxHead, softmaxHead)
)

In [ ]:
losses = {
	"classes": "categorical_crossentropy",
	"boxes": "mean_squared_error",
}

custom_model.compile(
    loss=losses,
    optimizer=optimizer,
    metrics=None,
)

print(custom_model.summary())

In [ ]:
custom_model.fit(
    train_ds.take(10),
    validation_data=val_ds.take(10),
    # Run for 10-35~ epochs to achieve good scores.
    epochs=1,
    #callbacks=[EvaluateCOCOMetricsCallback(val_ds.take(10))],
)